In [1]:
import optlang
import numpy as np
import sympy


from optlang import Model, Variable, Constraint, Objective

# All the (symbolic) variables are declared, with a name and optionally a lower and/or upper bound.
x1 = Variable('x1', lb=5, ub=100, type="integer")
x2 = Variable('x2', lb=0, ub=100, type="integer")
x3 = Variable('x3', lb=0, ub=100, type="integer")

# A constraint is constructed from an expression of variables and a lower and/or upper bound (lb and ub).
c1 = Constraint(x1 + x2 + x3, ub=100)
c2 = Constraint(10 * x1 + 4 * x2 + 5 * x3, ub=600)
c3 = Constraint(2 * x1 + 2 * x2 + 6 * x3, ub=300)

# An objective can be formulated
obj = Objective(10 * x1 + 6 * x2 + 4 * x3, direction='max')

# Variables, constraints and objective are combined in a Model object, which can subsequently be optimized.
model = Model(name='Simple model')
model.objective = obj
model.add([c1, c2, c3])

status = model.optimize()

print("status:", model.status)
print("objective value:", model.objective.value)
print("----------")
for var_name, var in model.variables.iteritems():
    print(var_name, "=", var.primal)

status: optimal
objective value: 732.0
----------
x1 = 33.0
x2 = 67.0
x3 = 0.0


In [9]:

def range_of_polynomial_with_bounded_vars(expr):
    expr = expr.expand()
    if not expr.is_polynomial():
        raise ValueError("The expression {} is not polynomial!".format(expr))

    min_v = 0
    max_v = 0

    # go through sum terms
    for summand in sympy.Add.make_args(expr):
        # get coefficient and product of variables
        c,vs = summand.as_coeff_Mul()
        
        min_fac = c
        max_fac = c
        # go through all factors
        for var,exp in vs.as_powers_dict().items():
            # get min and max value of factor
            val0 = var.lb**exp
            val1 = var.ub**exp
            new_vals= (min_fac*val0, min_fac*val1, max_fac*val0, max_fac*val1)
            min_fac = min(new_vals)
            max_fac = max(new_vals)
        
        min_v += min_fac
        max_v += max_fac
    
    return (min_v, max_v)
        

In [10]:
#    5 < x < 17
# -> x = 5 + x1 + 2x2 + 4x3 + 8x4, 
# -> 5 < 5 + x1 + 2x2 + 4x3 + 8x4 < 17 <=> 0 < x1 + 2x2 + 4x3 + 8x4 < 12

def binary_expansion(var: Variable):
    if var.type != "integer" and var.type != "binary":
        raise ValueError("Variable {} is not an integer, but this package can only handle integer variables!".format(var.name))
    
    # check if the variable has bounds - if so, add them as constraints, otherwise complain
    if var.lb is None:
        raise ValueError("Variable {} is does not have a defined lower bound!".format(var.name))
    if var.ub is None:
        raise ValueError("Variable {} is does not have a defined upper bound!".format(var.name))
    
    # compute value range and required number of bits to represent it
    num_bits = (int(np.ceil(np.log2(float(var.ub)-float(var.lb+1)))))
    
    # add binary variables
    sub_vars = []
    sub = var.lb
    for i in range(num_bits):
        # add the variable for the binary expansion
        sub_var = Variable("{}_{}".format(var.name, i), type="binary")
        sub_vars.append(sub_var)
        
        # add the monomial term to the expansion of the integer variable
        sub += (2**i) * sub_var
    
    # add the upper and lower bounds as a constraint on the binary expansion of the integer variable
    constraint = Constraint(sub, lb=var.lb, ub=var.ub)
    
    return sub_vars, sub, constraint

In [21]:
qubo_model = Model(name='QUBO model')
substitutions = dict()
variables = []
leq_constraints = []
zero_equations = []

# go through all variables
for var_name, var in model.variables.iteritems():
    
    # expand every non-binary variable
    if var.type != "binary":
        sub_vars, sub, constraint = binary_expansion(var)
        substitutions[var] = sub
        variables.extend(sub_vars)
        leq_constraints.append(constraint)
    else:
        variables.add(var)
    
# copy over all the explicit constraints, but substituting in the new variables and breaking them up into positive and negative constraints
for constraint in model.constraints:
    # substitute the expanded variables into the constraint expression
    expr = constraint.expression.subs(substitutions, simultaneous=False)
    
    # if both upper and lower bounds coincide, this is not an inequality constraint but rather an equation
    if constraint.ub is not None and constraint.lb is not None and constraint.ub==constraint.lb:
        # compute the value range this expression can take
        computed_lb, computed_ub = range_of_polynomial_with_bounded_vars(expr)    
    
        # check if it is even possible to satisfy this constraint, at all
        if not (computed_lb <= constraint.lb <= computed_ub):
            raise ValueError("Cannot satisfy constraint {}<={}<={} because {}<={}<={}".format(constraint.lb, constraint.expression, constraint.ub, computed_lb, expr, computed_ub))

        zero_equations.append(constraint.expression-constraint.ub)
    else:
        # otherwise, add the individual constraints to the list
        if constraint.ub is not None:
            leq_constraints.append(Constraint(expr, ub=constraint.ub))
        if constraint.lb is not None:
            leq_constraints.append(Constraint(-expr, ub=-constraint.lb))


s = 0
# introduce slack variables for all constraint equations
while len(leq_constraints) != 0:
    # take out a constraint
    constraint = leq_constraints.pop()
    
    # compute the value range this expression can take
    computed_lb, computed_ub = range_of_polynomial_with_bounded_vars(constraint.expression)    
    
    # check if it is even possible to satisfy this constraint, at all
    if computed_lb>constraint.ub:
        raise ValueError("Cannot satisfy constraint {}<={}<={} because {}>={}".format(computed_lb, constraint.expression, constraint.ub))

    # check if the constraint is trivial and can be ignored
    if computed_ub<=constraint.ub:
        print("Trivial! Contraint {} <= {} <= {}".format(constraint.expression, computed_ub, constraint.ub))
        continue

    #     constraint.expression <= constraint.ub
    # <=> constraint.expression - computed_lb + c <= constraint.ub - computed_lb + c
    # <=> constraint.expression - computed_lb + c = s;  0 <= s <= constraint.ub - computed_lb + c
    # <=> constraint.expression - computed_lb + c - s = 0; s <= constraint.ub - computed_lb + c
    
    # compute the next power of 2 for the rhs
    next_power_of_2 = (1<<int(np.ceil(np.log2(float(constraint.ub+1-computed_lb)))))-1
    
    # compute the offset needed to make the rhs a power of two
    c = next_power_of_2 - (constraint.ub - computed_lb)

    # do a binary expansion of the slack variable
    slack_vars, slack_eq, slack_constraint = binary_expansion(Variable("s_{}".format(s), type="integer", lb=0, ub=next_power_of_2))
    s+=1
    
    # add the variables
    variables.extend(slack_vars)
    
    # add the equation "constraint.expression + c - computed_lb - s = 0"; the inequality should now be trivial
    zero_equations.append(constraint.expression + c - computed_lb - slack_eq)


# copy over the objective, but with the new variables
objective = Objective((1 if model.objective.direction=="min" else -1)*model.objective.expression.subs(substitutions), direction="min")

In [22]:
for eq in zero_equations:
    print((eq**2).expand())

1.0*s_0_0**2 + 4.0*s_0_0*s_0_1 + 8.0*s_0_0*s_0_2 + 16.0*s_0_0*s_0_3 + 32.0*s_0_0*s_0_4 + 64.0*s_0_0*s_0_5 + 128.0*s_0_0*s_0_6 + 256.0*s_0_0*s_0_7 + 512.0*s_0_0*s_0_8 - 4.0*s_0_0*x1_0 - 8.0*s_0_0*x1_1 - 16.0*s_0_0*x1_2 - 32.0*s_0_0*x1_3 - 64.0*s_0_0*x1_4 - 128.0*s_0_0*x1_5 - 256.0*s_0_0*x1_6 - 4.0*s_0_0*x2_0 - 8.0*s_0_0*x2_1 - 16.0*s_0_0*x2_2 - 32.0*s_0_0*x2_3 - 64.0*s_0_0*x2_4 - 128.0*s_0_0*x2_5 - 256.0*s_0_0*x2_6 - 12.0*s_0_0*x3_0 - 24.0*s_0_0*x3_1 - 48.0*s_0_0*x3_2 - 96.0*s_0_0*x3_3 - 192.0*s_0_0*x3_4 - 384.0*s_0_0*x3_5 - 768.0*s_0_0*x3_6 - 442.0*s_0_0 + 4.0*s_0_1**2 + 16.0*s_0_1*s_0_2 + 32.0*s_0_1*s_0_3 + 64.0*s_0_1*s_0_4 + 128.0*s_0_1*s_0_5 + 256.0*s_0_1*s_0_6 + 512.0*s_0_1*s_0_7 + 1024.0*s_0_1*s_0_8 - 8.0*s_0_1*x1_0 - 16.0*s_0_1*x1_1 - 32.0*s_0_1*x1_2 - 64.0*s_0_1*x1_3 - 128.0*s_0_1*x1_4 - 256.0*s_0_1*x1_5 - 512.0*s_0_1*x1_6 - 8.0*s_0_1*x2_0 - 16.0*s_0_1*x2_1 - 32.0*s_0_1*x2_2 - 64.0*s_0_1*x2_3 - 128.0*s_0_1*x2_4 - 256.0*s_0_1*x2_5 - 512.0*s_0_1*x2_6 - 24.0*s_0_1*x3_0 - 48.0*s_0_1

In [127]:
Q = sympy.zeros(len(variables),len(variables))

# Add objective
lin_coeff = sympy.zeros(len(variables),1)
quad_coeff = sympy.zeros(len(variables),1)
for i,var in enumerate(variables):
    lin_coeff[i] = objective.expression.coeff(var, 1)
    quad_coeff[i] = objective.expression.coeff(var, 2)

# add linear terms, which have been artificially squared
#Q += sympy.diag(*lin_coeff) + sympy.diag(*quad_coeff)


penalty=1

# Add constraints
for eq in zero_equations:
    eq = eq.expand()
    tmp = sympy.zeros(len(variables),1)
    for i,var in enumerate(variables):
        tmp[i] = eq.coeff(var)
    
    c = 0
    for arg in eq.args:
        if arg.is_constant():
            c += arg
            
    # we are ignoring constant terms here, because they don't change the optimization problem
    # tmp[-1] = c
    
    # add quadratic terms
    Q += penalty*tmp*tmp.T
    
    # add linear terms, which have been artificially squared
    Q += penalty*2*c*sympy.diag(*tmp)

print(Q)

Matrix([[10578.0000000000, 212.000000000000, 424.000000000000, 848.000000000000, 1696.00000000000, 3392.00000000000, 6784.00000000000, 45.0000000000000, 90.0000000000000, 180.000000000000, 360.000000000000, 720.000000000000, 1440.00000000000, 2880.00000000000, 63.0000000000000, 126.000000000000, 252.000000000000, 504.000000000000, 1008.00000000000, 2016.00000000000, 4032.00000000000, -2.00000000000000, -4.00000000000000, -8.00000000000000, -16.0000000000000, -32.0000000000000, -64.0000000000000, -128.000000000000, -256.000000000000, -512.000000000000, -10.0000000000000, -20.0000000000000, -40.0000000000000, -80.0000000000000, -160.000000000000, -320.000000000000, -640.000000000000, -1280.00000000000, -2560.00000000000, -5120.00000000000, -1.00000000000000, -2.00000000000000, -4.00000000000000, -8.00000000000000, -16.0000000000000, -32.0000000000000, -64.0000000000000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -2, -4, -8, -16, -32, -64], [212.000000000000, 21368.0000000000, 848.0000

In [128]:
var_a = sympy.Array(variables, (len(variables),1))
(var_a.transpose() @ Q @ var_a)[0].expand()

-441.0*s_0_0**2 + 4*s_0_0*s_0_1 + 8*s_0_0*s_0_2 + 16*s_0_0*s_0_3 + 32*s_0_0*s_0_4 + 64*s_0_0*s_0_5 + 128*s_0_0*s_0_6 + 256*s_0_0*s_0_7 + 512*s_0_0*s_0_8 - 4.0*s_0_0*x1_0 - 8.0*s_0_0*x1_1 - 16.0*s_0_0*x1_2 - 32.0*s_0_0*x1_3 - 64.0*s_0_0*x1_4 - 128.0*s_0_0*x1_5 - 256.0*s_0_0*x1_6 - 4.0*s_0_0*x2_0 - 8.0*s_0_0*x2_1 - 16.0*s_0_0*x2_2 - 32.0*s_0_0*x2_3 - 64.0*s_0_0*x2_4 - 128.0*s_0_0*x2_5 - 256.0*s_0_0*x2_6 - 12.0*s_0_0*x3_0 - 24.0*s_0_0*x3_1 - 48.0*s_0_0*x3_2 - 96.0*s_0_0*x3_3 - 192.0*s_0_0*x3_4 - 384.0*s_0_0*x3_5 - 768.0*s_0_0*x3_6 - 880.0*s_0_1**2 + 16*s_0_1*s_0_2 + 32*s_0_1*s_0_3 + 64*s_0_1*s_0_4 + 128*s_0_1*s_0_5 + 256*s_0_1*s_0_6 + 512*s_0_1*s_0_7 + 1024*s_0_1*s_0_8 - 8.0*s_0_1*x1_0 - 16.0*s_0_1*x1_1 - 32.0*s_0_1*x1_2 - 64.0*s_0_1*x1_3 - 128.0*s_0_1*x1_4 - 256.0*s_0_1*x1_5 - 512.0*s_0_1*x1_6 - 8.0*s_0_1*x2_0 - 16.0*s_0_1*x2_1 - 32.0*s_0_1*x2_2 - 64.0*s_0_1*x2_3 - 128.0*s_0_1*x2_4 - 256.0*s_0_1*x2_5 - 512.0*s_0_1*x2_6 - 24.0*s_0_1*x3_0 - 48.0*s_0_1*x3_1 - 96.0*s_0_1*x3_2 - 192.0*s_0_1*x

In [131]:
(objective.expression + penalty*(sympy.Add(*(eq**2 for eq in zero_equations)))).expand()

1.0*s_0_0**2 + 4.0*s_0_0*s_0_1 + 8.0*s_0_0*s_0_2 + 16.0*s_0_0*s_0_3 + 32.0*s_0_0*s_0_4 + 64.0*s_0_0*s_0_5 + 128.0*s_0_0*s_0_6 + 256.0*s_0_0*s_0_7 + 512.0*s_0_0*s_0_8 - 4.0*s_0_0*x1_0 - 8.0*s_0_0*x1_1 - 16.0*s_0_0*x1_2 - 32.0*s_0_0*x1_3 - 64.0*s_0_0*x1_4 - 128.0*s_0_0*x1_5 - 256.0*s_0_0*x1_6 - 4.0*s_0_0*x2_0 - 8.0*s_0_0*x2_1 - 16.0*s_0_0*x2_2 - 32.0*s_0_0*x2_3 - 64.0*s_0_0*x2_4 - 128.0*s_0_0*x2_5 - 256.0*s_0_0*x2_6 - 12.0*s_0_0*x3_0 - 24.0*s_0_0*x3_1 - 48.0*s_0_0*x3_2 - 96.0*s_0_0*x3_3 - 192.0*s_0_0*x3_4 - 384.0*s_0_0*x3_5 - 768.0*s_0_0*x3_6 - 442.0*s_0_0 + 4.0*s_0_1**2 + 16.0*s_0_1*s_0_2 + 32.0*s_0_1*s_0_3 + 64.0*s_0_1*s_0_4 + 128.0*s_0_1*s_0_5 + 256.0*s_0_1*s_0_6 + 512.0*s_0_1*s_0_7 + 1024.0*s_0_1*s_0_8 - 8.0*s_0_1*x1_0 - 16.0*s_0_1*x1_1 - 32.0*s_0_1*x1_2 - 64.0*s_0_1*x1_3 - 128.0*s_0_1*x1_4 - 256.0*s_0_1*x1_5 - 512.0*s_0_1*x1_6 - 8.0*s_0_1*x2_0 - 16.0*s_0_1*x2_1 - 32.0*s_0_1*x2_2 - 64.0*s_0_1*x2_3 - 128.0*s_0_1*x2_4 - 256.0*s_0_1*x2_5 - 512.0*s_0_1*x2_6 - 24.0*s_0_1*x3_0 - 48.0*s_0_1

In [132]:
Q

Matrix([
[10578.0,    212.0,    424.0,    848.0,   1696.0,    3392.0,    6784.0,    45.0,    90.0,   180.0,    360.0,    720.0,   1440.0,    2880.0,    63.0,   126.0,    252.0,    504.0,   1008.0,   2016.0,    4032.0,   -2.0,   -4.0,    -8.0,   -16.0,   -32.0,    -64.0,   -128.0,   -256.0,   -512.0,  -10.0,   -20.0,   -40.0,   -80.0,   -160.0,   -320.0,   -640.0,   -1280.0,   -2560.0,   -5120.0,  -1.0,   -2.0,   -4.0,   -8.0,   -16.0,   -32.0,   -64.0,   0,    0,    0,    0,     0,     0,     0,   0,    0,    0,    0,     0,     0,     0,    -1,     -2,     -4,     -8,    -16,     -32,   -64],
[  212.0,  21368.0,    848.0,   1696.0,   3392.0,    6784.0,   13568.0,    90.0,   180.0,   360.0,    720.0,   1440.0,   2880.0,    5760.0,   126.0,   252.0,    504.0,   1008.0,   2016.0,   4032.0,    8064.0,   -4.0,   -8.0,   -16.0,   -32.0,   -64.0,   -128.0,   -256.0,   -512.0,  -1024.0,  -20.0,   -40.0,   -80.0,  -160.0,   -320.0,   -640.0,  -1280.0,   -2560.0,   -5120.0,  -10240.0,  -2.0,   